In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

verif_data = c[c['date'].isin([f"{date.today()}"])]


if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
c.head(3)

Autosave disabled
Not today, check other link


,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.tail(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
17292,2020-05-13,Zimbabwe,0,0,37,4
17293,2020-05-14,Zimbabwe,0,0,37,4
17294,2020-05-15,Zimbabwe,0,0,37,4


In [3]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-15,France,141915.0,0.0,27622.0,0.0,0.0,0.0
0,2020-05-15,China,84105.0,0.0,4658.0,0.0,0.0,0.0
0,2020-05-15,Italy,224038.0,0.0,31571.0,0.0,0.0,0.0
0,2020-05-15,Spain,229948.0,0.0,27496.0,0.0,0.0,0.0
0,2020-05-15,United States,1440429.0,0.0,87679.0,0.0,0.0,0.0
0,2020-05-15,World,4499233.0,0.0,307646.0,0.0,0.0,0.0
0,2020-05-15,United Kingdom,236718.0,0.0,34078.0,0.0,0.0,0.0
0,2020-05-15,Germany,174103.0,0.0,7917.0,0.0,0.0,0.0
0,2020-05-15,Iran,116516.0,0.0,6912.0,0.0,0.0,0.0
0,2020-05-15,Turkey,146270.0,0.0,4060.0,0.0,0.0,0.0


In [5]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [7]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-15,France,141915.0,141356.0,27622.0,27425.0,559.0,197.0
0,2020-05-15,China,84105.0,84029.0,4658.0,4637.0,76.0,21.0
0,2020-05-15,Italy,224038.0,223096.0,31571.0,31368.0,942.0,203.0
0,2020-05-14,Spain,229948.0,229540.0,27496.0,27321.0,408.0,175.0
0,2020-05-15,United States,1440429.0,1417889.0,87679.0,85906.0,22540.0,1773.0
0,2020-05-15,World,4499233.0,4405680.0,307646.0,302115.0,93553.0,5531.0
0,2020-05-15,United Kingdom,236718.0,233151.0,34078.0,33614.0,3567.0,464.0
0,2020-05-15,Germany,174103.0,173152.0,7917.0,7824.0,951.0,93.0
0,2020-05-15,Iran,116516.0,114533.0,6912.0,6854.0,1983.0,58.0
0,2020-05-15,Turkey,146270.0,144749.0,4060.0,4007.0,1521.0,53.0


In [8]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")